In [1]:
import nltk
from nltk import pos_tag
from nltk.tokenize import word_tokenize
from textblob import TextBlob
import pandas as pd

In [2]:
atepc_data = pd.read_json('../data/atepc-result.json')

In [3]:
atepc_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3678 entries, 0 to 3677
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   sentence    3678 non-null   object
 1   IOB         3678 non-null   object
 2   tokens      3678 non-null   object
 3   aspect      3678 non-null   object
 4   position    3678 non-null   object
 5   sentiment   3678 non-null   object
 6   probs       3678 non-null   object
 7   confidence  3678 non-null   object
dtypes: object(8)
memory usage: 230.0+ KB


In [4]:
reviews = atepc_data['sentence']
reviews = reviews.tolist()

In [5]:
len(reviews)

3678

In [6]:
train_data = reviews[:1300]
test_data = reviews[1300:]

In [7]:
train_set = pd.DataFrame(train_data)

In [8]:
test_set = pd.DataFrame(test_data)

In [9]:
train_set.to_csv('../data/train_set.csv')

In [10]:
def remove_pattern(pattern, text):
    new_text = text.re(pattern)

In [11]:
import emoji
import re

In [12]:

pattern = r'[^\w\s]'
clean_reviews = []
for comment in reviews:
    text = emoji.demojize(comment)
    new_text = re.sub(pattern, '', text)
    clean_reviews.append(new_text)

In [13]:
import pyabsa
from pyabsa import ATEPCCheckpointManager

No CUDA GPU found in your device


/bin/sh: nvidia-smi: command not found
/Users/josephineamponsah/Documents/project-folder/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Version 0.9.7 of metric_visualizer is outdated. Version 0.9.9 was released Tuesday July 04, 2023.
[2023-07-20 15:46:09] (2.3.1) PyABSA(2.3.1): If your code crashes on Colab, please use the GPU runtime. Then run "pip install pyabsa[dev] -U" and restart the kernel.
Or if it does not work, you can use v1.16.27

[New Feature] Aspect Sentiment Triplet Extraction since v2.1.0 (https://github.com/yangheng95/PyABSA/tree/v2/examples-v2/aspect_sentiment_triplet_extration)
[New Feature] Aspect CategoryOpinion Sentiment Quadruple Extraction since v2.2.0 (https://github.com/yangheng95/PyABSA/tree/v2/examples-v2/aspect_opinion_sentiment_category_extraction)



In [14]:
aspect_extractor = ATEPCCheckpointManager.get_aspect_extractor(checkpoint='english',
                                   auto_device=False )


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/pool.py:265: ResourceWarning: unclosed running multiprocessing pool <multiprocessing.pool.Pool state=RUN pool_size=1>
  _warn(f"unclosed running multiprocessing pool {self!r}",


[2023-07-20 15:46:19] (2.3.1) Downloading checkpoint:english 
[2023-07-20 15:46:19] (2.3.1) Notice: The pretrained model are used for testing, it is recommended to train the model on your own custom datasets
[2023-07-20 15:46:19] (2.3.1) Checkpoint already downloaded, skip
[2023-07-20 15:46:19] (2.3.1) Load aspect extractor from checkpoints/ATEPC_ENGLISH_CHECKPOINT/fast_lcf_atepc_English_cdw_apcacc_82.36_apcf1_81.89_atef1_75.43
[2023-07-20 15:46:19] (2.3.1) config: checkpoints/ATEPC_ENGLISH_CHECKPOINT/fast_lcf_atepc_English_cdw_apcacc_82.36_apcf1_81.89_atef1_75.43/fast_lcf_atepc.config
[2023-07-20 15:46:19] (2.3.1) state_dict: checkpoints/ATEPC_ENGLISH_CHECKPOINT/fast_lcf_atepc_English_cdw_apcacc_82.36_apcf1_81.89_atef1_75.43/fast_lcf_atepc.state_dict
[2023-07-20 15:46:19] (2.3.1) model: None
[2023-07-20 15:46:19] (2.3.1) tokenizer: checkpoints/ATEPC_ENGLISH_CHECKPOINT/fast_lcf_atepc_English_cdw_apcacc_82.36_apcf1_81.89_atef1_75.43/fast_lcf_atepc.tokenizer
[2023-07-20 15:46:19] (2.3.1)

Some weights of the model checkpoint at microsoft/deberta-v3-base were not used when initializing DebertaV2Model: ['lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.classifier.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.bias', 'mask_predictions.dense.bias', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.LayerNorm.bias', 'mask_predictions.dense.weight', 'mask_predictions.classifier.bias', 'mask_predictions.LayerNorm.weight']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Special tokens have b

In [15]:
inference_source = clean_reviews
atepc_result = aspect_extractor.extract_aspect(inference_source=inference_source,  #
                          pred_sentiment=True,  # Predict the sentiment of extracted aspect terms
                          )

preparing apc inference dataloader: 100%|██████████| 6547/6547 [00:08<00:00, 755.51it/s] 
/Users/josephineamponsah/Documents/project-folder/.venv/lib/python3.9/site-packages/pyabsa/tasks/AspectTermExtraction/prediction/aspect_extractor.py:567: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:248.)
  lcf_cdm_vec = torch.tensor(
classifying aspect sentiments:   0%|          | 0/205 [00:00<?, ?it/s]/Users/josephineamponsah/Documents/project-folder/.venv/lib/python3.9/site-packages/pyabsa/tasks/AspectTermExtraction/prediction/aspect_extractor.py:648: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  float(x) for x in F.softmax(i_apc_logits).cpu().numpy().tolist()
classifying aspe

[2023-07-20 16:35:39] (2.3.1) The results of aspect term extraction have been saved in /Users/josephineamponsah/Documents/project-folder/notebooks/Aspect Term Extraction and Polarity Classification.FAST_LCF_ATEPC.result.json
[2023-07-20 16:35:39] (2.3.1) Example 0: I had a wonderful experience at Jonat Hotel Their <rooms:Positive Confidence:0.9988> are new and their <services:Positive Confidence:0.9995> are top notch I definitely recommend it to anyone visiting Cape Coast folded _ hands _ dark _ skin _ tone
[2023-07-20 16:35:39] (2.3.1) Example 1: I enjoyed my <stay:Positive Confidence:0.9995> here For a serene <environment:Positive Confidence:0.9994> choose Jonat Hotel
[2023-07-20 16:35:39] (2.3.1) Example 2: The couple of <times:Positive Confidence:0.9985> I have been there were very peaceful times and I really enjoyed the serenity and the calm <ambiance:Positive Confidence:0.9992> of the place Really nice <rooms:Positive Confidence:0.999> and affordable as well
[2023-07-20 16:35:39]

In [16]:
result = pd.DataFrame(atepc_result)
    

In [17]:
import json
with open("../data/absa-result.json", 'w') as file:
    json.dump(atepc_result, file)

In [18]:
from pyabsa import make_ABSA_dataset

In [19]:
testing = reviews[500:1200]

In [20]:
train = make_ABSA_dataset(dataset_name_or_path = '../data/train_set.csv', checkpoint= 'english')

[2023-07-20 16:35:40] (2.3.1) Downloading checkpoint:english 
[2023-07-20 16:35:40] (2.3.1) Notice: The pretrained model are used for testing, it is recommended to train the model on your own custom datasets
[2023-07-20 16:35:40] (2.3.1) Checkpoint already downloaded, skip
[2023-07-20 16:35:40] (2.3.1) Load aspect extractor from checkpoints/ATEPC_ENGLISH_CHECKPOINT/fast_lcf_atepc_English_cdw_apcacc_82.36_apcf1_81.89_atef1_75.43
[2023-07-20 16:35:40] (2.3.1) config: checkpoints/ATEPC_ENGLISH_CHECKPOINT/fast_lcf_atepc_English_cdw_apcacc_82.36_apcf1_81.89_atef1_75.43/fast_lcf_atepc.config
[2023-07-20 16:35:40] (2.3.1) state_dict: checkpoints/ATEPC_ENGLISH_CHECKPOINT/fast_lcf_atepc_English_cdw_apcacc_82.36_apcf1_81.89_atef1_75.43/fast_lcf_atepc.state_dict
[2023-07-20 16:35:40] (2.3.1) model: None
[2023-07-20 16:35:40] (2.3.1) tokenizer: checkpoints/ATEPC_ENGLISH_CHECKPOINT/fast_lcf_atepc_English_cdw_apcacc_82.36_apcf1_81.89_atef1_75.43/fast_lcf_atepc.tokenizer
[2023-07-20 16:35:40] (2.3.1)

Some weights of the model checkpoint at microsoft/deberta-v3-base were not used when initializing DebertaV2Model: ['lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.classifier.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.bias', 'mask_predictions.dense.bias', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.LayerNorm.bias', 'mask_predictions.dense.weight', 'mask_predictions.classifier.bias', 'mask_predictions.LayerNorm.weight']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Special tokens have b

[2023-07-20 16:35:44] (2.3.1) Start processing dataset: ../data/train_set.csv


classifying aspect sentiments: 100%|██████████| 75/75 [11:54:31<00:00, 571.62s/it]    


[2023-07-21 07:06:39] (2.3.1) The results of aspect term extraction have been saved in /Users/josephineamponsah/Documents/project-folder/notebooks/Aspect Term Extraction and Polarity Classification.FAST_LCF_ATEPC.result.json
[2023-07-21 07:06:39] (2.3.1) Example 0: , 0
[2023-07-21 07:06:39] (2.3.1) Example 1: 0 , I had a wonderful experience at Jonat Hotel . Their <rooms:Positive Confidence:0.9995> are new and their <services:Positive Confidence:0.9995> are top - notch . I definitely recommend it to anyone visiting Cape Coast 🙏🏿
[2023-07-21 07:06:39] (2.3.1) Example 2: 1 , " I enjoyed my <stay:Positive Confidence:0.9995> here . For a serene <environment:Positive Confidence:0.9995> , choose Jonat Hotel . "
[2023-07-21 07:06:39] (2.3.1) Example 3: 2 , " The couple of <times:Positive Confidence:0.9989> I have been there , were very peaceful times and I really enjoyed the serenity and the calm <ambiance:Positive Confidence:0.9991> of the place . Really nice <rooms:Positive Confidence:0.999

In [21]:
from pyabsa import AspectSentimentTripletExtraction as ASTE

In [22]:
config = (
    ASTE.ASTEConfigManager.get_aste_config_english()
) 
config.model = ASTE.ASTEModelList.EMCGCN 

In [23]:
from pyabsa import ModelSaveOption, DeviceTypeOption
import warnings

warnings.filterwarnings("ignore")

config.batch_size = 16
config.patience = 999
config.log_step = -1
config.seed = [1, 2, 3]
config.verbose = False  # If verbose == True, PyABSA will output the model structure and several processed data examples
config.notice = (
    "This is an training example for aspect term extraction"  # for memos usage
)

trainer = ASTE.ASTETrainer(
    config=config,
    dataset = ASTE.ASTEDatasetList.Restaurant14,
    # from_checkpoint="english",  # if you want to resume training from our pretrained checkpoints, you can pass the checkpoint name here
    auto_device=DeviceTypeOption.AUTO,  # use cuda if available
    checkpoint_save_mode=ModelSaveOption.SAVE_MODEL_STATE_DICT,  # save state dict only instead of the whole model
    load_aug=False,  # there are some augmentation dataset for integrated datasets, you use them by setting load_aug=True to improve performance
)

[2023-07-21 07:06:39] (2.3.1) Set Model Device: cpu
[2023-07-21 07:06:39] (2.3.1) Device Name: Unknown
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
2023-07-21 07:06:50,062 INFO: PyABSA version: 2.3.1
2023-07-21 07:06:50,086 INFO: Transformers version: 4.30.2
2023-07-21 07:06:50,089 INFO: Torch version: 2.0.1+cudaNone
2023-07-21 07:06:50,089 INFO: Device: Unknown
2023-07-21 07:06:50,090 INFO: Restaurant14 in the trainer is not a exact path, will search dataset in current working directory
2023-07-21 07:06:50,144 INFO: You can set load_aug=True in a trainer to augment your dataset (English only yet) and improve performance.
2023-07-21 07:06:50,162 INFO: Load cache dataset from emcgcn.Restaurant14.dataset.83ceb0936

Some weights of the model checkpoint at yangheng/deberta-v3-base-absa-v1.1 were not used when initializing DebertaV2Model: ['pooler.dense.weight', 'pooler.dense.bias', 'classifier.bias', 'classifier.weight']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[2023-07-21 07:07:00] (2.3.1) ABSADatasetsVersion:None	-->	Calling Count:0
[2023-07-21 07:07:00] (2.3.1) MV:<metric_visualizer.metric_visualizer.MetricVisualizer object at 0x1576b8d30>	-->	Calling Count:0
[2023-07-21 07:07:00] (2.3.1) PyABSAVersion:2.3.1	-->	Calling Count:1
[2023-07-21 07:07:00] (2.3.1) SRD:3	-->	Calling Count:0
[2023-07-21 07:07:00] (2.3.1) TorchVersion:2.0.1+cudaNone	-->	Calling Count:1
[2023-07-21 07:07:00] (2.3.1) TransformersVersion:4.30.2	-->	Calling Count:1
[2023-07-21 07:07:00] (2.3.1) adam_epsilon:1e-08	-->	Calling Count:1
[2023-07-21 07:07:00] (2.3.1) auto_device:True	-->	Calling Count:4
[2023-07-21 07:07:00] (2.3.1) batch_size:16	-->	Calling Count:0
[2023-07-21 07:07:00] (2.3.1) cache_dataset:True	-->	Calling Count:1
[2023-07-21 07:07:00] (2.3.1) checkpoint_save_mode:1	-->	Calling Count:4
[2023-07-21 07:07:00] (2.3.1) cross_validate_fold:-1	-->	Calling Count:0
[2023-07-21 07:07:00] (2.3.1) dataset_file:{'train': ['integrated_datasets/aste_datasets/400.SemEva

Epoch:  0 | Smooth Loss: 0.5601: 100%|██████████| 80/80 [06:20<00:00,  4.75s/it, Dev F1:23.10(max:23.10)]
Epoch:  1 | Smooth Loss: 0.4534: 100%|██████████| 80/80 [06:16<00:00,  4.70s/it, Dev F1:35.90(max:35.90)]
Epoch:  2 | Smooth Loss: 0.3999: 100%|██████████| 80/80 [3:19:27<00:00, 149.60s/it, Dev F1:36.49(max:36.49)]
Epoch:  3 | Smooth Loss: 0.3638: 100%|██████████| 80/80 [32:25<00:00, 24.32s/it, Dev F1:44.37(max:44.37)]
Epoch:  4 | Smooth Loss: 0.3363: 100%|██████████| 80/80 [06:14<00:00,  4.68s/it, Dev F1:54.48(max:54.48)]
Epoch:  5 | Smooth Loss: 0.3146: 100%|██████████| 80/80 [06:10<00:00,  4.63s/it, Dev F1:59.52(max:59.52)]
Epoch:  6 | Smooth Loss: 0.2956: 100%|██████████| 80/80 [06:36<00:00,  4.95s/it, Dev F1:58.52(max:59.52)]
Epoch:  7 | Smooth Loss: 0.2793: 100%|██████████| 80/80 [06:34<00:00,  4.93s/it, Dev F1:55.78(max:59.52)]
Epoch:  8 | Smooth Loss: 0.2654: 100%|██████████| 80/80 [07:27<00:00,  5.60s/it, Dev F1:58.57(max:59.52)]
Epoch:  9 | Smooth Loss: 0.2533: 100%|█████

[2023-07-21 11:50:58] (2.3.1) Loading best model: checkpoints/emcgcn_Restaurant14_f1_59.52/ and evaluating on test set 
2023-07-21 11:51:16,928 INFO: 
----------------------------------------------------------- Raw Metric Records -----------------------------------------------------------
╒═════════════╤════════════════════════════════════════════════════════╤══════════╤═══════════╤══════════╤═══════╤═══════╤═══════╤═══════╕
│ Metric      │ Trial                                                  │ Values   │  Average  │  Median  │  Std  │  IQR  │  Min  │  Max  │
╞═════════════╪════════════════════════════════════════════════════════╪══════════╪═══════════╪══════════╪═══════╪═══════╪═══════╪═══════╡
│ Max-Test-F1 │ emcgcn-Restaurant14-yangheng/deberta-v3-base-absa-v1.1 │ [59.52]  │   59.52   │  59.52   │   0   │   0   │ 59.52 │ 59.52 │
╘═════════════╧════════════════════════════════════════════════════════╧══════════╧═══════════╧══════════╧═══════╧═══════╧═══════╧═══════╛
---------------

...

[2023-07-21 12:58:23] (2.3.1) Loading best model: checkpoints/emcgcn_Restaurant14_f1_64.17/ and evaluating on test set 
2023-07-21 12:58:40,904 INFO: 
----------------------------------------------------------- Raw Metric Records -----------------------------------------------------------
╒═════════════╤════════════════════════════════════════════════════════╤══════════╤═══════════╤══════════╤═══════╤═══════╤═══════╤═══════╕
│ Metric      │ Trial                                                  │ Values   │  Average  │  Median  │  Std  │  IQR  │  Min  │  Max  │
╞═════════════╪════════════════════════════════════════════════════════╪══════════╪═══════════╪══════════╪═══════╪═══════╪═══════╪═══════╡
│ Max-Test-F1 │ emcgcn-Restaurant14-yangheng/deberta-v3-base-absa-v1.1 │ [64.17]  │   64.17   │  64.17   │   0   │   0   │ 64.17 │ 64.17 │
╘═════════════╧════════════════════════════════════════════════════════╧══════════╧═══════════╧══════════╧═══════╧═══════╧═══════╧═══════╛
---------------

Some weights of the model checkpoint at yangheng/deberta-v3-base-absa-v1.1 were not used when initializing DebertaV2Model: ['pooler.dense.weight', 'pooler.dense.bias', 'classifier.bias', 'classifier.weight']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[2023-07-21 12:58:55] (2.3.1) ABSADatasetsVersion:None	-->	Calling Count:0
[2023-07-21 12:58:55] (2.3.1) MV:<metric_visualizer.metric_visualizer.MetricVisualizer object at 0x2a91b1a00>	-->	Calling Count:0
[2023-07-21 12:58:55] (2.3.1) PyABSAVersion:2.3.1	-->	Calling Count:1
[2023-07-21 12:58:55] (2.3.1) SRD:3	-->	Calling Count:0
[2023-07-21 12:58:55] (2.3.1) TorchVersion:2.0.1+cudaNone	-->	Calling Count:1
[2023-07-21 12:58:55] (2.3.1) TransformersVersion:4.30.2	-->	Calling Count:1
[2023-07-21 12:58:55] (2.3.1) adam_epsilon:1e-08	-->	Calling Count:1
[2023-07-21 12:58:55] (2.3.1) auto_device:True	-->	Calling Count:4
[2023-07-21 12:58:55] (2.3.1) batch_size:16	-->	Calling Count:0
[2023-07-21 12:58:55] (2.3.1) cache_dataset:True	-->	Calling Count:1
[2023-07-21 12:58:55] (2.3.1) checkpoint_save_mode:1	-->	Calling Count:4
[2023-07-21 12:58:55] (2.3.1) cross_validate_fold:-1	-->	Calling Count:0
[2023-07-21 12:58:55] (2.3.1) dataset_file:{'train': ['integrated_datasets/aste_datasets/400.SemEva

Epoch:  0 | Smooth Loss: 0.5250: 100%|██████████| 80/80 [06:39<00:00,  4.99s/it, Dev F1:1.45(max:1.45)]
Epoch:  1 | Smooth Loss: 0.4359: 100%|██████████| 80/80 [06:35<00:00,  4.94s/it, Dev F1:33.65(max:33.65)]
Epoch:  2 | Smooth Loss: 0.3870: 100%|██████████| 80/80 [06:37<00:00,  4.97s/it, Dev F1:36.68(max:36.68)]
Epoch:  3 | Smooth Loss: 0.3537: 100%|██████████| 80/80 [06:48<00:00,  5.11s/it, Dev F1:34.80(max:36.68)]
Epoch:  4 | Smooth Loss: 0.3284: 100%|██████████| 80/80 [07:40<00:00,  5.76s/it, Dev F1:47.79(max:47.79)]
Epoch:  5 | Smooth Loss: 0.3070: 100%|██████████| 80/80 [40:23<00:00, 30.30s/it, Dev F1:53.75(max:53.75)]
Epoch:  6 | Smooth Loss: 0.2881: 100%|██████████| 80/80 [07:23<00:00,  5.54s/it, Dev F1:60.71(max:60.71)]
Epoch:  7 | Smooth Loss: 0.2715: 100%|██████████| 80/80 [07:24<00:00,  5.56s/it, Dev F1:62.19(max:62.19)]
Epoch:  8 | Smooth Loss: 0.2574: 100%|██████████| 80/80 [07:13<00:00,  5.42s/it, Dev F1:62.51(max:62.51)]
Epoch:  9 | Smooth Loss: 0.2451: 100%|██████████

[2023-07-21 14:42:59] (2.3.1) Loading best model: checkpoints/emcgcn_Restaurant14_f1_62.51/ and evaluating on test set 
2023-07-21 14:43:19,039 INFO: 
----------------------------------------------------------- Raw Metric Records -----------------------------------------------------------
╒═════════════╤════════════════════════════════════════════════════════╤══════════╤═══════════╤══════════╤═══════╤═══════╤═══════╤═══════╕
│ Metric      │ Trial                                                  │ Values   │  Average  │  Median  │  Std  │  IQR  │  Min  │  Max  │
╞═════════════╪════════════════════════════════════════════════════════╪══════════╪═══════════╪══════════╪═══════╪═══════╪═══════╪═══════╡
│ Max-Test-F1 │ emcgcn-Restaurant14-yangheng/deberta-v3-base-absa-v1.1 │ [62.51]  │   62.51   │  62.51   │   0   │   0   │ 62.51 │ 62.51 │
╘═════════════╧════════════════════════════════════════════════════════╧══════════╧═══════════╧══════════╧═══════╧═══════╧═══════╧═══════╛
---------------

In [24]:
aspect_triplet_extractor = trainer.load_trained_model()

[2023-07-21 14:43:22] (2.3.1) Load sentiment classifier from checkpoints/emcgcn_Restaurant14_f1_64.17/
[2023-07-21 14:43:22] (2.3.1) config: checkpoints/emcgcn_Restaurant14_f1_64.17/emcgcn.config
[2023-07-21 14:43:22] (2.3.1) state_dict: checkpoints/emcgcn_Restaurant14_f1_64.17/emcgcn.state_dict
[2023-07-21 14:43:22] (2.3.1) model: None
[2023-07-21 14:43:22] (2.3.1) tokenizer: checkpoints/emcgcn_Restaurant14_f1_64.17/emcgcn.tokenizer
[2023-07-21 14:43:23] (2.3.1) Set Model Device: cpu
[2023-07-21 14:43:23] (2.3.1) Device Name: Unknown


Some weights of the model checkpoint at yangheng/deberta-v3-base-absa-v1.1 were not used when initializing DebertaV2Model: ['pooler.dense.weight', 'pooler.dense.bias', 'classifier.bias', 'classifier.weight']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [25]:
output = aspect_triplet_extractor.predict(reviews[0])

[2023-07-21 14:43:28] (2.3.1) Batch: 0 {'sentence_id': 0, 'sentence': 'I had a wonderful experience at Jonat Hotel . Their rooms are new and their services are top - notch . I definitely recommend it to anyone visiting Cape Coast 🙏🏿', 'Triplets': [{'Aspect': 'rooms', 'Opinion': 'wonderful', 'Polarity': 'Positive'}, {'Aspect': 'rooms', 'Opinion': 'new', 'Polarity': 'Positive'}, {'Aspect': 'rooms', 'Opinion': 'top', 'Polarity': 'Positive'}, {'Aspect': 'services', 'Opinion': 'wonderful', 'Polarity': 'Positive'}, {'Aspect': 'services', 'Opinion': 'new', 'Polarity': 'Positive'}, {'Aspect': 'services', 'Opinion': 'top', 'Polarity': 'Positive'}], 'True Triplets': []}
[2023-07-21 14:43:28] (2.3.1) Batch: 0 {'sentence_id': 0, 'sentence': 'I had a wonderful experience at Jonat Hotel . Their rooms are new and their services are top - notch . I definitely recommend it to anyone visiting Cape Coast 🙏🏿', 'Triplets': [{'Aspect': 'rooms', 'Opinion': 'wonderful', 'Polarity': 'Positive'}, {'Aspect': 'ro

In [26]:
output

{'sentence_id': 0,
 'sentence': 'I had a wonderful experience at Jonat Hotel . Their rooms are new and their services are top - notch . I definitely recommend it to anyone visiting Cape Coast 🙏🏿',
 'Triplets': [{'Aspect': 'rooms',
   'Opinion': 'wonderful',
   'Polarity': 'Positive'},
  {'Aspect': 'rooms', 'Opinion': 'new', 'Polarity': 'Positive'},
  {'Aspect': 'rooms', 'Opinion': 'top', 'Polarity': 'Positive'},
  {'Aspect': 'services', 'Opinion': 'wonderful', 'Polarity': 'Positive'},
  {'Aspect': 'services', 'Opinion': 'new', 'Polarity': 'Positive'},
  {'Aspect': 'services', 'Opinion': 'top', 'Polarity': 'Positive'}],
 'True Triplets': []}

In [27]:

test_output = []
for record in reviews[500:550]:
    test_output.append(aspect_triplet_extractor.predict(record))

[2023-07-21 14:43:28] (2.3.1) Batch: 0 {'sentence_id': 0, 'sentence': "It ' s a very cool place to have a holiday", 'Triplets': [{'Aspect': 'place', 'Opinion': 'cool', 'Polarity': 'Positive'}], 'True Triplets': []}
[2023-07-21 14:43:28] (2.3.1) Batch: 0 {'sentence_id': 0, 'sentence': "It ' s a very cool place to have a holiday", 'Triplets': [{'Aspect': 'place', 'Opinion': 'cool', 'Polarity': 'Positive'}], 'True Triplets': []}
[2023-07-21 14:43:28] (2.3.1) Batch: 0 {'sentence_id': 0, 'sentence': 'beautifully endowed with a calm and quite atmosphere . definitely one of the perfect weekend getaway .', 'Triplets': [{'Aspect': 'atmosphere', 'Opinion': 'calm', 'Polarity': 'Positive'}, {'Aspect': 'atmosphere', 'Opinion': 'quite', 'Polarity': 'Positive'}], 'True Triplets': []}
[2023-07-21 14:43:28] (2.3.1) Batch: 0 {'sentence_id': 0, 'sentence': 'beautifully endowed with a calm and quite atmosphere . definitely one of the perfect weekend getaway .', 'Triplets': [{'Aspect': 'atmosphere', 'Opini

In [28]:
test_output

[{'sentence_id': 0,
  'sentence': "It ' s a very cool place to have a holiday",
  'Triplets': [{'Aspect': 'place', 'Opinion': 'cool', 'Polarity': 'Positive'}],
  'True Triplets': []},
 {'sentence_id': 0,
  'sentence': 'beautifully endowed with a calm and quite atmosphere . definitely one of the perfect weekend getaway .',
  'Triplets': [{'Aspect': 'atmosphere',
    'Opinion': 'calm',
    'Polarity': 'Positive'},
   {'Aspect': 'atmosphere', 'Opinion': 'quite', 'Polarity': 'Positive'}],
  'True Triplets': []},
 {'sentence_id': 0,
  'sentence': 'Amazing remove place for weekend getaway , during low tide the bay is very pleasant to swim in .',
  'Triplets': [{'Aspect': 'place',
    'Opinion': 'Amazing',
    'Polarity': 'Positive'},
   {'Aspect': 'place', 'Opinion': 'pleasant', 'Polarity': 'Positive'},
   {'Aspect': 'bay', 'Opinion': 'Amazing', 'Polarity': 'Positive'},
   {'Aspect': 'bay', 'Opinion': 'pleasant', 'Polarity': 'Positive'}],
  'True Triplets': []},
 {'sentence_id': 0,
  'senten

In [29]:
from pyabsa import download_all_available_datasets

download_all_available_datasets()

[2023-07-21 14:43:35] (2.3.1) Datasets already exist in /Users/josephineamponsah/Documents/project-folder/notebooks/integrated_datasets, skip download


In [30]:
from pyabsa import DatasetItem
my_dataset = DatasetItem("reviews", ["my_dataset1", "my_dataset2"])
# my_dataset1 and my_dataset2 are the dataset folders. In there folders, the train dataset is necessary

In [31]:
from pyabsa import AspectTermExtraction as ATEPC 
dataset = ATEPC.ATEPCDatasetList.Restaurant14
from pyabsa import ModelSaveOption, DeviceTypeOption
import warnings
config = (
    ATEPC.ATEPCConfigManager.get_atepc_config_english()
)  # this config contains 'pretrained_bert', it is based on pretrained models
config.model = ATEPC.ATEPCModelList.FAST_LCF_ATEPC  # improved version of LCF-ATEPC
warnings.filterwarnings("ignore")

config.batch_size = 16
# config.patience = 2
# config.log_step = -1
config.seed = [1]
config.verbose = True  # If verbose == True, PyABSA will output the model strcture and seversal processed data examples
config.notice = (
    "This is an training example for aspect term extraction"  # for memos usage
)

trainer = ATEPC.ATEPCTrainer(
    config=config,
    dataset=dataset,
    from_checkpoint="english",  # if you want to resume training from our pretrained checkpoints, you can pass the checkpoint name here
    auto_device=DeviceTypeOption.AUTO,  # use cuda if available
    checkpoint_save_mode=ModelSaveOption.SAVE_MODEL_STATE_DICT,  # save state dict only instead of the whole model
    load_aug=False,  # there are some augmentation dataset for integrated datasets, you use them by setting load_aug=True to improve performance
)

[2023-07-21 14:43:35] (2.3.1) Set Model Device: cpu
[2023-07-21 14:43:35] (2.3.1) Device Name: Unknown


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
2023-07-21 14:43:35,602 INFO: PyABSA version: 2.3.1
2023-07-21 14:43:35,604 INFO: Transformers version: 4.30.2
2023-07-21 14:43:35,604 INFO: Torch version: 2.0.1+cudaNone
2023-07-21 14:43:35,605 INFO: Device: Unknown
2023-07-21 14:43:35,605 INFO: Restaurant14 in the trainer is not a exact path, will search dataset in current working directory
2023-07-21 14:43:35,637 INFO: You can set load_aug=True in a trainer to augment your dataset (English only yet) and improve performance.


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


2023-07-21 14:43:37,465 INFO: Load cache dataset from fast_lcf_atepc.Restaurant14.dataset.4f297fbe3d40f8a5f1b7666967a009a2a1d08213c23069709b663d443de772b6.cache


Some weights of the model checkpoint at microsoft/deberta-v3-base were not used when initializing DebertaV2Model: ['lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.classifier.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.bias', 'mask_predictions.dense.bias', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.LayerNorm.bias', 'mask_predictions.dense.weight', 'mask_predictions.classifier.bias', 'mask_predictions.LayerNorm.weight']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


2023-07-21 14:43:39,692 INFO: ABSADatasetsVersion:None	-->	Calling Count:0
2023-07-21 14:43:39,693 INFO: IOB_label_to_index:{'B-ASP': 1, 'I-ASP': 2, 'O': 3, '[CLS]': 4, '[SEP]': 5}	-->	Calling Count:1
2023-07-21 14:43:39,693 INFO: MV:<metric_visualizer.metric_visualizer.MetricVisualizer object at 0x2a75a4850>	-->	Calling Count:0
2023-07-21 14:43:39,693 INFO: PyABSAVersion:2.3.1	-->	Calling Count:1
2023-07-21 14:43:39,694 INFO: SRD:3	-->	Calling Count:9444
2023-07-21 14:43:39,694 INFO: TorchVersion:2.0.1+cudaNone	-->	Calling Count:1
2023-07-21 14:43:39,694 INFO: TransformersVersion:4.30.2	-->	Calling Count:1
2023-07-21 14:43:39,694 INFO: auto_device:True	-->	Calling Count:3
2023-07-21 14:43:39,695 INFO: batch_size:16	-->	Calling Count:7
2023-07-21 14:43:39,695 INFO: cache_dataset:True	-->	Calling Count:1
2023-07-21 14:43:39,695 INFO: checkpoint_save_mode:1	-->	Calling Count:4
2023-07-21 14:43:39,695 INFO: cross_validate_fold:-1	-->	Calling Count:0
2023-07-21 14:43:39,696 INFO: dataset_f

Epoch:  0 | Smooth Loss: 0.1011: 100%|██████████| 226/226 [26:39<00:00,  7.08s/it,  APC_ACC: 87.03(max:88.37) | APC_F1: 81.00(max:82.82) | ATE_F1: 82.56(max:85.74)]                
Epoch:  1 | Smooth Loss: 0.0815: 100%|██████████| 226/226 [26:40<00:00,  7.08s/it,  APC_ACC: 87.39(max:88.37) | APC_F1: 81.82(max:82.82) | ATE_F1: 83.22(max:85.74)]                
Epoch:  2 | Smooth Loss: 0.0760: 100%|██████████| 226/226 [36:45<00:00,  9.76s/it,  APC_ACC: 87.39(max:88.37) | APC_F1: 80.71(max:82.82) | ATE_F1: 80.52(max:85.74)]                
Epoch:  3 | Smooth Loss: 0.0696: 100%|██████████| 226/226 [28:51<00:00,  7.66s/it,  APC_ACC: 88.01(max:88.37) | APC_F1: 81.96(max:82.82) | ATE_F1: 84.03(max:85.74)]                
Epoch:  4 | Smooth Loss: 0.0695: 100%|██████████| 226/226 [27:10<00:00,  7.22s/it,  APC_ACC: 87.12(max:88.37) | APC_F1: 80.32(max:82.82) | ATE_F1: 86.00(max:86.48)]                
Epoch:  5 | Smooth Loss: 0.0667: 100%|██████████| 226/226 [2:37:32<00:00, 41.82s/it,  APC_ACC: 

2023-07-21 22:39:31,874 INFO: 
-------------------------------------------------------------------- Raw Metric Records --------------------------------------------------------------------
╒════════════════════════════════╤═══════════════════════════════════════════════════════╤══════════╤═══════════╤══════════╤═══════╤═══════╤═══════╤═══════╕
│ Metric                         │ Trial                                                 │ Values   │  Average  │  Median  │  Std  │  IQR  │  Min  │  Max  │
╞════════════════════════════════╪═══════════════════════════════════════════════════════╪══════════╪═══════════╪══════════╪═══════╪═══════╪═══════╪═══════╡
│ Max-APC-Test-Acc w/o Valid Set │ fast_lcf_atepc-Restaurant14-microsoft/deberta-v3-base │ [88.82]  │   88.82   │  88.82   │   0   │   0   │ 88.82 │ 88.82 │
├────────────────────────────────┼───────────────────────────────────────────────────────┼──────────┼───────────┼──────────┼───────┼───────┼───────┼───────┤
│ Max-APC-Test-F1 w/o Valid

In [32]:
atepc_model = trainer.load_trained_model()

[2023-07-21 22:39:35] (2.3.1) Load aspect extractor from checkpoints/fast_lcf_atepc_Restaurant14_cdw_apcacc_88.82_apcf1_83.3_atef1_83.52/
[2023-07-21 22:39:35] (2.3.1) config: checkpoints/fast_lcf_atepc_Restaurant14_cdw_apcacc_88.82_apcf1_83.3_atef1_83.52/fast_lcf_atepc.config
[2023-07-21 22:39:35] (2.3.1) state_dict: checkpoints/fast_lcf_atepc_Restaurant14_cdw_apcacc_88.82_apcf1_83.3_atef1_83.52/fast_lcf_atepc.state_dict
[2023-07-21 22:39:35] (2.3.1) model: None
[2023-07-21 22:39:35] (2.3.1) tokenizer: checkpoints/fast_lcf_atepc_Restaurant14_cdw_apcacc_88.82_apcf1_83.3_atef1_83.52/fast_lcf_atepc.tokenizer
[2023-07-21 22:39:35] (2.3.1) Set Model Device: cpu
[2023-07-21 22:39:35] (2.3.1) Device Name: Unknown


Some weights of the model checkpoint at microsoft/deberta-v3-base were not used when initializing DebertaV2Model: ['lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.classifier.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.bias', 'mask_predictions.dense.bias', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.LayerNorm.bias', 'mask_predictions.dense.weight', 'mask_predictions.classifier.bias', 'mask_predictions.LayerNorm.weight']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Special tokens have b

In [33]:
ate_result =  []
for record in reviews[500:550]:
    ate_result.append(atepc_model.predict(record))


[2023-07-21 22:39:42] (2.3.1) The results of aspect term extraction have been saved in /Users/josephineamponsah/Documents/project-folder/notebooks/Aspect Term Extraction and Polarity Classification.FAST_LCF_ATEPC.result.json
[2023-07-21 22:39:42] (2.3.1) Example 0: It ' s a very cool <place:Positive Confidence:0.9997> to have a holiday
[2023-07-21 22:39:43] (2.3.1) The results of aspect term extraction have been saved in /Users/josephineamponsah/Documents/project-folder/notebooks/Aspect Term Extraction and Polarity Classification.FAST_LCF_ATEPC.result.json
[2023-07-21 22:39:43] (2.3.1) Example 0: beautifully endowed with a calm and quite <atmosphere:Positive Confidence:0.9997> . definitely one of the perfect weekend getaway .
[2023-07-21 22:39:44] (2.3.1) The results of aspect term extraction have been saved in /Users/josephineamponsah/Documents/project-folder/notebooks/Aspect Term Extraction and Polarity Classification.FAST_LCF_ATEPC.result.json
[2023-07-21 22:39:44] (2.3.1) Example 0

In [34]:
ate_result

[{'sentence': "It ' s a very cool place to have a holiday",
  'IOB': ['O', 'O', 'O', 'O', 'O', 'O', 'B-ASP', 'O', 'O', 'O', 'O'],
  'tokens': ['It',
   "'",
   's',
   'a',
   'very',
   'cool',
   'place',
   'to',
   'have',
   'a',
   'holiday'],
  'aspect': ['place'],
  'position': [[6]],
  'sentiment': ['Positive'],
  'probs': [[7.689416088396683e-05,
    0.00025777192786335945,
    0.999665379524231]],
  'confidence': [0.9997]},
 {'sentence': 'beautifully endowed with a calm and quite atmosphere . definitely one of the perfect weekend getaway .',
  'IOB': ['O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'B-ASP',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O'],
  'tokens': ['beautifully',
   'endowed',
   'with',
   'a',
   'calm',
   'and',
   'quite',
   'atmosphere',
   '.',
   'definitely',
   'one',
   'of',
   'the',
   'perfect',
   'weekend',
   'getaway',
   '.'],
  'aspect': ['atmosphere'],
  'position': [[7]],
  'sentiment': ['Positive'],

In [35]:
from pyabsa.tasks.ABSAInstruction import dataset_list

In [36]:
dataset_list

<module 'pyabsa.tasks.ABSAInstruction.dataset_list' from '/Users/josephineamponsah/Documents/project-folder/.venv/lib/python3.9/site-packages/pyabsa/tasks/ABSAInstruction/dataset_list.py'>

In [37]:
from pyabsa.tasks.ABSAInstruction.instruction import CategoryInstruction

In [38]:
categorize = CategoryInstruction()
check = categorize.prepare_input(reviews[554], ['site', 'food'])

In [39]:
from pyabsa import ABSAInstruction

In [40]:
quadruple_extractor = ABSAInstruction.ABSAGenerator("multilingual")

[2023-07-21 22:40:26] (2.3.1) Downloading checkpoint:multilingual 
[2023-07-21 22:40:26] (2.3.1) Notice: The pretrained model are used for testing, it is recommended to train the model on your own custom datasets
[2023-07-21 22:40:26] (2.3.1) Checkpoint already downloaded, skip


In [41]:
quadruple_extractor.predict(reviews[554])

{'text': "I am simply posting here again because you can ' t respond to a post once the owner has posted . I just want to clarify that there is no way the previous response was accurate because it referred to me receiving a discount because my father was arriving from cape coast . My father passed away some time back , I came with my friend , and we certainly did not receive a discount . The toilet seat was dirty , there ' s no way around that , I cleaned it . I strongly feel in the service industry there is no room for arrogance but for humility and proving the best service . We did not receive that at Safari Beach Lodge . The kitchen staff was friendly but the food was late , no arguing that , even ask them because we went to ask for the food two times and they apologized both times . There was a group of 5 gentlemen also staying in the hotel so maybe that delayed out food . My main annoyance had come from one of the staff that seemed relaxed about the fact that our room was not read

{'text': "I am simply posting here again because you can ' t respond to a post once the owner has posted . I just want to clarify that there is no way the previous response was accurate because it referred to me receiving a discount because my father was arriving from cape coast . My father passed away some time back , I came with my friend , and we certainly did not receive a discount . The toilet seat was dirty , there ' s no way around that , I cleaned it . I strongly feel in the service industry there is no room for arrogance but for humility and proving the best service . We did not receive that at Safari Beach Lodge . The kitchen staff was friendly but the food was late , no arguing that , even ask them because we went to ask for the food two times and they apologized both times . There was a group of 5 gentlemen also staying in the hotel so maybe that delayed out food . My main annoyance had come from one of the staff that seemed relaxed about the fact that our room was not read

In [42]:
config4 = (
    ABSAInstruction
)

In [43]:
quad_model = ABSAInstruction.model.T5Generator('multilingual')

[2023-07-21 22:40:33] (2.3.1) Downloading checkpoint:multilingual 
[2023-07-21 22:40:33] (2.3.1) Notice: The pretrained model are used for testing, it is recommended to train the model on your own custom datasets
[2023-07-21 22:40:33] (2.3.1) Checkpoint already downloaded, skip


In [44]:
dataset = ABSAInstruction.ACOSDatasetList.Restaurant14

In [45]:
input_data = 'integrated_datasets/acos_datasets/502.Restaurant14/train.jsonl'

In [46]:
train_data = pd.read_json(r'integrated_datasets/acos_datasets/502.Restaurant14/train.jsonl', lines= True)

In [47]:
import json

In [48]:
train_file  = []
with open(input_data, 'r') as file:
    for line in file:
        train_file.append(json.loads(line))


In [49]:
train_file

[{'text': 'But the staff was so horrible to us .',
  'labels': [{'aspect': 'staff',
    'opinion': 'horrible',
    'polarity': 'negative',
    'category': 'NULL'}]},
 {'text': "To be completely fair , the only redeeming factor was the food , which was above average , but could n't make up for all the other deficiencies of Teodora .",
  'labels': [{'aspect': 'food',
    'opinion': 'above average',
    'polarity': 'positive',
    'category': 'NULL'}]},
 {'text': "The food is uniformly exceptional , with a very capable kitchen which will proudly whip up whatever you feel like eating , whether it 's on the menu or not .",
  'labels': [{'aspect': 'food',
    'opinion': 'exceptional',
    'polarity': 'positive',
    'category': 'NULL'},
   {'aspect': 'kitchen',
    'opinion': 'capable',
    'polarity': 'positive',
    'category': 'NULL'}]},
 {'text': 'Our agreed favorite is the orrechiete with sausage and chicken ( usually the waiters are kind enough to split the dish in half so you get to s

In [50]:
with open(input_data, 'r') as file:
    for line in file:
        # Parse the line as a JSON object (Python dictionary)
        json_object = json.loads(line)

In [51]:
json_object

{'text': 'Each table has a pot of boiling water sunken into its surface , and you get platters of thin sliced meats , various vegetables , and rice and glass noodles .',
 'labels': [{'aspect': 'meats',
   'opinion': 'thin',
   'polarity': 'neutral',
   'category': 'NULL'},
  {'aspect': 'vegetables',
   'opinion': 'various',
   'polarity': 'neutral',
   'category': 'NULL'}]}

In [52]:
def load_jsonl_file(file_path):
    json_objects = []
    with open(file_path, 'r') as file:
        for line in file:
            json_objects.append(json.loads(line))
    return json_objects

# Load the JSONL file and parse it into a list of JSON objects
json_objects_list = load_jsonl_file(input_data)

In [53]:
json_objects_list

[{'text': 'But the staff was so horrible to us .',
  'labels': [{'aspect': 'staff',
    'opinion': 'horrible',
    'polarity': 'negative',
    'category': 'NULL'}]},
 {'text': "To be completely fair , the only redeeming factor was the food , which was above average , but could n't make up for all the other deficiencies of Teodora .",
  'labels': [{'aspect': 'food',
    'opinion': 'above average',
    'polarity': 'positive',
    'category': 'NULL'}]},
 {'text': "The food is uniformly exceptional , with a very capable kitchen which will proudly whip up whatever you feel like eating , whether it 's on the menu or not .",
  'labels': [{'aspect': 'food',
    'opinion': 'exceptional',
    'polarity': 'positive',
    'category': 'NULL'},
   {'aspect': 'kitchen',
    'opinion': 'capable',
    'polarity': 'positive',
    'category': 'NULL'}]},
 {'text': 'Our agreed favorite is the orrechiete with sausage and chicken ( usually the waiters are kind enough to split the dish in half so you get to s

In [58]:
from pyabsa.tasks.ABSAInstruction.instruction import CategoryInstruction, Instruction

In [59]:
test = Instruction("I really love the food here")

In [60]:
categorize = CategoryInstruction(test)

In [61]:
categorize

In [ ]:
# for sample in json_objects_list:
tokenized_data = quad_model.tokenize_function_inputs(sample= json_objects_list)

In [ ]:
trained_quad = quad_model.train(tokenized_datasets = dataset, output_dir = 'notebooks/')